In [ ]:
# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print('✓ Bibliotecas importadas correctamente')

## 1. Carga y Exploración de Datos

In [ ]:
# Cargar datos
df = pd.read_csv('../datasets/viabilidad_acuicultura.csv')

print('Dimensiones del dataset:', df.shape)
print('\nPrimeras filas:')
df.head()

In [ ]:
# Información del dataset
print('Información del dataset:')
df.info()

print('\nEstadísticas descriptivas:')
df.describe()

In [ ]:
# Distribución de la variable objetivo
print('Distribución de sitios viables vs no viables:')
print(df['viable'].value_counts())
print('\nPorcentaje:')
print(df['viable'].value_counts(normalize=True) * 100)

# Visualización
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

df['viable'].value_counts().plot(kind='bar', ax=ax[0], color=['#e74c3c', '#2ecc71'])
ax[0].set_title('Distribución de Viabilidad', fontsize=14, fontweight='bold')
ax[0].set_xlabel('Viable (0=No, 1=Sí)')
ax[0].set_ylabel('Frecuencia')
ax[0].set_xticklabels(['No Viable', 'Viable'], rotation=0)

df['viable'].value_counts().plot(kind='pie', ax=ax[1], autopct='%1.1f%%', 
                                  colors=['#e74c3c', '#2ecc71'], startangle=90)
ax[1].set_title('Proporción de Viabilidad', fontsize=14, fontweight='bold')
ax[1].set_ylabel('')

plt.tight_layout()
plt.show()

## 2. Análisis Exploratorio de Datos (EDA)

In [ ]:
# Matriz de correlación para variables numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            square=True, linewidths=1, cbar_kws={'shrink': 0.8})
plt.title('Matriz de Correlación - Variables Numéricas', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# Distribución de variables numéricas por viabilidad
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

numeric_features = ['temperatura_C', 'salinidad_ppt', 'oxigeno_mg_L', 'pH', 'profundidad_m', 'distancia_costa_km']

for idx, col in enumerate(numeric_features):
    df.boxplot(column=col, by='viable', ax=axes[idx], patch_artist=True)
    axes[idx].set_title(f'{col} por Viabilidad', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Viable (0=No, 1=Sí)')
    axes[idx].set_ylabel(col)
    plt.sca(axes[idx])
    plt.xticks([1, 2], ['No Viable', 'Viable'])

plt.suptitle('')
plt.tight_layout()
plt.show()

## 3. Preprocesamiento de Datos

In [ ]:
# Codificar variables categóricas
le_exposicion = LabelEncoder()
le_fondo = LabelEncoder()

df['exposicion_oleaje_encoded'] = le_exposicion.fit_transform(df['exposicion_oleaje'])
df['tipo_fondo_encoded'] = le_fondo.fit_transform(df['tipo_fondo'])

print('Codificación de exposición al oleaje:')
print(dict(zip(le_exposicion.classes_, le_exposicion.transform(le_exposicion.classes_))))
print('\nCodificación de tipo de fondo:')
print(dict(zip(le_fondo.classes_, le_fondo.transform(le_fondo.classes_))))

In [ ]:
# Seleccionar características para el modelo
features = ['temperatura_C', 'salinidad_ppt', 'oxigeno_mg_L', 'pH', 
            'profundidad_m', 'distancia_costa_km', 'exposicion_oleaje_encoded', 'tipo_fondo_encoded']

X = df[features]
y = df['viable']

print('Forma de X:', X.shape)
print('Forma de y:', y.shape)

In [ ]:
# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print('Datos de entrenamiento:', X_train.shape)
print('Datos de prueba:', X_test.shape)
print('\nDistribución en entrenamiento:')
print(y_train.value_counts())
print('\nDistribución en prueba:')
print(y_test.value_counts())

In [ ]:
# Estandarizar características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('✓ Datos estandarizados')
print('\nMedia de características entrenamiento (después de escalar):')
print(np.mean(X_train_scaled, axis=0))
print('\nDesviación estándar de características entrenamiento (después de escalar):')
print(np.std(X_train_scaled, axis=0))

## 4. Entrenamiento del Modelo de Regresión Logística

In [ ]:
# Crear y entrenar el modelo
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X_train_scaled, y_train)

print('✓ Modelo entrenado exitosamente')
print('\nCoeficientes del modelo:')
coef_df = pd.DataFrame({
    'Característica': features,
    'Coeficiente': log_reg.coef_[0]
}).sort_values('Coeficiente', key=abs, ascending=False)

print(coef_df)
print(f'\nIntercept: {log_reg.intercept_[0]:.4f}')

In [ ]:
# Visualizar importancia de características
plt.figure(figsize=(10, 6))
colors = ['green' if c > 0 else 'red' for c in coef_df['Coeficiente']]
plt.barh(coef_df['Característica'], coef_df['Coeficiente'], color=colors, alpha=0.7)
plt.xlabel('Coeficiente', fontsize=12)
plt.title('Importancia de Características (Coeficientes del Modelo)', fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='--', linewidth=1)
plt.tight_layout()
plt.show()

print('\n📊 Interpretación:')
print('- Coeficientes positivos: aumentan la probabilidad de viabilidad')
print('- Coeficientes negativos: disminuyen la probabilidad de viabilidad')

## 5. Evaluación del Modelo

In [ ]:
# Predicciones
y_pred_train = log_reg.predict(X_train_scaled)
y_pred_test = log_reg.predict(X_test_scaled)

y_pred_proba_train = log_reg.predict_proba(X_train_scaled)[:, 1]
y_pred_proba_test = log_reg.predict_proba(X_test_scaled)[:, 1]

# Accuracy
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

print('Accuracy en Entrenamiento:', f'{train_accuracy:.4f}')
print('Accuracy en Prueba:', f'{test_accuracy:.4f}')

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred_test)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', square=True, 
            xticklabels=['No Viable', 'Viable'],
            yticklabels=['No Viable', 'Viable'],
            cbar_kws={'shrink': 0.8})
plt.ylabel('Valor Real', fontsize=12)
plt.xlabel('Predicción', fontsize=12)
plt.title('Matriz de Confusión', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Reporte de clasificación
print('Reporte de Clasificación - Conjunto de Prueba:')
print(classification_report(y_test, y_pred_test, target_names=['No Viable', 'Viable']))

In [ ]:
# Curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba_test)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Clasificador Aleatorio')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (FPR)', fontsize=12)
plt.ylabel('Tasa de Verdaderos Positivos (TPR)', fontsize=12)
plt.title('Curva ROC - Receiver Operating Characteristic', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f'\nÁrea bajo la curva ROC (AUC): {roc_auc:.4f}')

## 6. Predicción en Nuevos Datos

In [ ]:
# Ejemplo de predicción para un nuevo sitio
nuevo_sitio = pd.DataFrame({
    'temperatura_C': [11.5],
    'salinidad_ppt': [33.8],
    'oxigeno_mg_L': [8.4],
    'pH': [8.1],
    'profundidad_m': [17],
    'distancia_costa_km': [2.3],
    'exposicion_oleaje': ['baja'],
    'tipo_fondo': ['rocoso']
})

# Codificar variables categóricas
nuevo_sitio['exposicion_oleaje_encoded'] = le_exposicion.transform(nuevo_sitio['exposicion_oleaje'])
nuevo_sitio['tipo_fondo_encoded'] = le_fondo.transform(nuevo_sitio['tipo_fondo'])

# Seleccionar características
nuevo_sitio_features = nuevo_sitio[features]

# Escalar
nuevo_sitio_scaled = scaler.transform(nuevo_sitio_features)

# Predecir
prediccion = log_reg.predict(nuevo_sitio_scaled)[0]
probabilidad = log_reg.predict_proba(nuevo_sitio_scaled)[0]

print('🎯 Predicción para el nuevo sitio:')
print('=' * 50)
print(f'Viabilidad: {"VIABLE" if prediccion == 1 else "NO VIABLE"}')
print(f'Probabilidad de NO viabilidad: {probabilidad[0]:.2%}')
print(f'Probabilidad de viabilidad: {probabilidad[1]:.2%}')
print('=' * 50)

## 7. Conclusiones

### Resultados del Modelo
- El modelo de regresión logística logra una buena separación entre sitios viables y no viables
- Las variables más importantes incluyen: oxígeno disuelto, temperatura y exposición al oleaje
- El modelo puede utilizarse como herramienta preliminar de evaluación de sitios

### Aplicaciones Prácticas
- **Selección de sitios**: Identificar ubicaciones potencialmente viables antes de realizar estudios costosos
- **Gestión ambiental**: Monitorear cambios en las condiciones que afectan la viabilidad
- **Planificación**: Apoyar decisiones sobre inversiones en acuicultura marina

### Limitaciones
- Los datos sintéticos utilizados son ilustrativos; se requieren datos reales para aplicaciones operativas
- El modelo no considera factores económicos, regulatorios ni sociales
- Se recomienda validación con expertos del sector acuícola

### Próximos Pasos
1. Incorporar datos reales de proyectos acuícolas existentes
2. Incluir variables adicionales (corrientes, especies objetivo, infraestructura)
3. Probar otros algoritmos (Random Forest, SVM) para comparar rendimiento
4. Desarrollar una interfaz web para facilitar el uso del modelo por profesionales del sector